This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [2]:
# First, import the relevant modules

import requests
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url ='https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
req = requests.get(url)
json_data = req.json()

In [4]:
afx_2017 = json_data['dataset_data']
print(type(afx_2017))

<class 'dict'>


In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json_data['dataset_data']['column_names'])
print(json_data['dataset_data']['data'][0])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [6]:
print(json_data['dataset_data'].keys())

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [7]:
json_data['dataset_data']['data'][0]  # First entry of the data

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [8]:
# 3. Calculate highest and lowest opening prices

# Extract the data from the dictionary as a list of lists, where each list is a row.
data_2017 = afx_2017['data']

# list comprehenstion to extract values for opens column from each row. ignore all None values.
opens = [i[1] for i in data_2017 if i[1] is not None]

highest_open = max(opens)
lowest_open = min(opens)
print(f'Highest opening = ${highest_open:.2f}. \nLowest opening = ${lowest_open:.2f}.')

Highest opening = $53.11. 
Lowest opening = $34.00.


In [9]:
# 4. Calculate largest change in any one day

# list comprehension to generate a list of the difference between the 3rd and 4th element in each list
largest_change = max([i[2] - i[3] for i in data_2017])
print(f'Largest change = ${largest_change:.2f}.')

Largest change = $2.81.


In [10]:
# 5. Calculate the largest closing price change between any 2 days.

# list comprehension to create a list of all closing values
closes = [i[4] for i in data_2017]

# list comprehension to calculate the difference between a value in closes.
overnight_changes = [val - closes[key-1] for key, val in enumerate(closes) 
                     if val is not None 
                     and closes[key-1] is not None 
                     and key > 0]

# Find the maximum overnight increase and decrease.
positive_max_overnight = max(overnight_changes)
negative_max_overnight = min(overnight_changes)

# Check if the largest difference is positive or negative and print the result.
# use abs() method to calculate the absolute value 
if abs(negative_max_overnight) > positive_max_overnight: 
    print(f'The largest overnight change = ${abs(negative_max_overnight):.2f} decrease')
else:
    print(f'The largest overnight change = ${positive_max_overnight:.2f} increase')

The largest overnight change = $2.56 increase


In [11]:
# 6. Find average daily trading volume.

# list comprehension to generate a list of the traded volume items.

def mean(x):
    return sum(x)/len(x)
traded_volume = [i[6] for i in data_2017 if i[6] is not None]
average_volume = mean(traded_volume)

print(f' Mean = {average_volume:.2f}')

 Mean = 89124.34


In [12]:
# 7.What was the median trading volume during this year

def median(v):
    n = len(v)
    sorted_v = sorted(v)
    midpoint = n//2
    if n % 2 == 1:
        #if odd return the median value
        return sorted_v[midpoint]
    else:
        # if even return the average of the middle value
        lo = midpoint - 1
        hi = midpoint
        return(sorted_v[lo]+sorted_v[hi])/2
med_volume = median(traded_volume)
print(f' Median = {med_volume:.2f}')

 Median = 76286.00
